In [1]:
# Dependencies and Setup
import pandas as pd
import csv
import numpy as np
from geopy.geocoders import Nominatim


# EXTRACT

In [2]:
# File to Load
crime_to_load = "Resources/Baltimore_2019_Crime_Only.csv"
cctv_to_load = "Resources/Baltimore_CCTV_Locations.csv"

In [3]:
# Read the CSV files
crime_df = pd.read_csv(crime_to_load)
cctv_df = pd.read_csv(cctv_to_load)

In [4]:
crime_df.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Weapon,District,Neighborhood,Total Incidents
0,07/06/2019,01:30:00,5B,0 HILLSIDE RD,BURGLARY,NaN,NORTHERN,ROLAND PARK,1
1,07/06/2019,09:00:00,7A,1600 APPLETON ST,AUTO THEFT,NaN,WESTERN,EASTERWOOD,1
2,07/06/2019,07:03:00,6E,2500 EDGECOMBE CIR N,LARCENY,NaN,NORTHERN,PARKLANE,1
3,07/06/2019,02:30:00,3AK,PL & N HIGHLAND AV,ROBBERY - STREET,KNIFE,SOUTHEAST,NaN,1
4,07/06/2019,10:05:00,6C,700 WASHINGTON BLVD,LARCENY,NaN,SOUTHERN,WASHINGTON VILLAGE,1


In [5]:
cctv_df.head()

,cameraLocation,cameraNumber,cameraProject,Location 1
0,Eutaw and Lexington Market,1,Downtown,"(39.290996, -76.621073999999993)"
1,Eutaw and Fayette,2,Downtown,"(39.29048796517317, -76.623665143680341)"
2,Eutaw and Baltimore,3,Downtown,"(39.289324000000001, -76.620985000000005)"
3,Eutaw and Redwood,4,Downtown,"(39.288677999999997, -76.620947999999999)"
4,Eutaw and Lombard,5,Downtown,"(39.287528000000002, -76.620853999999994)"


# TRANSFORM

In [6]:
# Changing "Location 1" form cctv_df to 'latitude' and 'longitiude'
# This is required for geocoding which we want to use to find addresses of deployed CCTV

# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in cctv_df['Location 1']:
    # Try to,
    try:
        #Remove ( from the string
        row = row.replace("(", "")
        row = row.replace(")", "")
        # Split the row by comma, convert to float, and append
        # everything before the comma to lat
        lat.append(float(row.split(',')[0]))
        # Split the row by comma, convert to float, and append
        # everything after the comma to lon
        lon.append(float(row.split(',')[1]))
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
cctv_df['latitude'] = lat
cctv_df['longitude'] = lon

In [7]:
cctv_df.head()

,cameraLocation,cameraNumber,cameraProject,Location 1,latitude,longitude
0,Eutaw and Lexington Market,1,Downtown,"(39.290996, -76.621073999999993)",39.290996,-76.621074
1,Eutaw and Fayette,2,Downtown,"(39.29048796517317, -76.623665143680341)",39.290488,-76.623665
2,Eutaw and Baltimore,3,Downtown,"(39.289324000000001, -76.620985000000005)",39.289324,-76.620985
3,Eutaw and Redwood,4,Downtown,"(39.288677999999997, -76.620947999999999)",39.288678,-76.620948
4,Eutaw and Lombard,5,Downtown,"(39.287528000000002, -76.620853999999994)",39.287528,-76.620854


### Test geolocator

In [46]:
# Test how does geolocator.reverse works
geolocator = Nominatim(user_agent="Baltimore_Crime")
location = geolocator.reverse('39.290996, -76.621074')

In [47]:
location.address

'King Of Lexington Jewelry, 110, North Eutaw Street, Seton Hill, Baltimore, Maryland, 21201, USA'

In [21]:
lon_lat_list = list(zip(cctv_df['latitude'], cctv_df['longitude']))
len(lon_lat_list)

603

In [29]:
str(lon_lat_list[0]).replace('(', '').replace(')', '')

'39.290996, -76.621074'

In [49]:
def convert_to_address(row) -> str:
#     print(row)
    lat = row['latitude']
    lon = row['longitude']
    input_lon_lat = str(lat) + ',' + str(lon)
    geolocator = Nominatim(user_agent="Baltimore_Crime")
    location = geolocator.reverse(input_lon_lat)
    print(location.address)
    return location.address

In [50]:
sample = lon_lat_list[0]
sample

(39.290996, -76.621074)

In [ ]:
cctv['address'] = cctv_df.apply(lambda row: convert_to_address(row), axis = 1)
cctv.head()

King Of Lexington Jewelry, 110, North Eutaw Street, Seton Hill, Baltimore, Maryland, 21201, USA
University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA
Medusa, 401, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
400, West Redwood Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
The Goddess, West Lombard Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
South Eutaw Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
511, West Pratt Street, Ridgleys Delight, Baltimore, Maryland, 21201, USA
University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA
University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA
University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA
601, West Lexington Street, Seton Hill, Baltimore, Maryland, 21201, USA
500, West Faye

Eastern Avenue, Fells Point, Baltimore, Maryland, 21231, USA
423, South Ann Street, Fells Prospect, Baltimore, Maryland, 21231, USA
South Broadway & Fleet Street, South Broadway, Fells Point, Baltimore, Maryland, 21231, USA
1704, Fleet Street, Fells Point, Baltimore, Maryland, 21231, USA
536, South Ann Street, Fells Prospect, Baltimore, Maryland, 21231, USA
1442, Fleet Street, Harbor East, Baltimore, Maryland, 21231, USA
600, South Eden Street, Harbor East, Baltimore, Maryland, 21205, USA
1703, Aliceanna Street, Fells Point, Baltimore, Maryland, 21231, USA
1615, Aliceanna Street, Fells Point, Baltimore, Maryland, 21231, USA
The Admiral's Cup, 1647, Thames Street, Fells Point, Baltimore, Maryland, 21231, USA
2776, Berea Road, Cherry Hill, Baltimore, Maryland, 21225, USA
1210, Round Road, Cherry Hill, Baltimore, Maryland, 21225, USA
1399, Slater Road, Cherry Hill, Baltimore, Maryland, 21225, USA
2705, Berea Road, Cherry Hill, Baltimore, Maryland, 21225, USA
1211, Shellbanks Road, Cherry 

2510, McElderry Street, McElderry Park, Baltimore, Maryland, 21205, USA
2541, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
2620, East Madison Street, McElderry Park, Baltimore, Maryland, 21205, USA
2642, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
601, North Belnord Avenue, McElderry Park, Baltimore, Maryland, 21205, USA
2739, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
800, North Streeper Street, McElderry Park, Baltimore, Maryland, 21205, USA
2901, McElderry Street, Ellwood Park/Monument, Baltimore, Maryland, 21205, USA
700, North Curley Street, Baltimore - Linwood, Baltimore, Maryland, 21205, USA
3000, McElderry Street, Ellwood Park/Monument, Baltimore, Maryland, 21205, USA
3021, East Monument Street, McElderry Park, Baltimore, Maryland, 21205, USA
3027, East Madison Street, Ellwood Park/Monument, Baltimore, Maryland, 21205, USA
3118, East Monument Street, Ellwood Park/Monument, Baltimore, Maryland, 21205, 

2752, Ellicott Driveway, Winchester, Baltimore, Maryland, 21216, USA
1027, Poplar Grove Street, Winchester, Baltimore, Maryland, 21216, USA
2800, Mosher Street, Mosher, Baltimore, Maryland, 21216, USA
Alexander Hamilton Elementary School, West Lafayette Avenue, Franklintown Road, Baltimore, Maryland, 21216, USA
719, North Longwood Street, Franklintown Road, Baltimore, Maryland, 21216, USA
714, Poplar Grove Street, Winchester, Baltimore, Maryland, 21216, USA
2838, Edmondson Avenue, Mosher, Baltimore, Maryland, 21223, USA
2594, Edmondson Avenue, Mosher, Baltimore, Maryland, 21223, USA
2444, Edmondson Avenue, Mosher, Baltimore, Maryland, 21223, USA
3332, Edmondson Avenue, Allendale, Baltimore, Maryland, 21229, USA
601, Swann Avenue, Rognel Heights, Baltimore, Maryland, 21229, USA
3940, Edmondson Avenue, Edmondson Village, Baltimore, Maryland, 21229, USA
3624, Edmondson Avenue, Allendale, Baltimore, Maryland, 21229, USA
3037, Presstman Street, Rosemont, Baltimore, Maryland, 21216, USA
1107

In [14]:
for row in cctv_df:
    print(row)

cameraLocation
cameraNumber
cameraProject
Location 1
latitude
longitude


### Translate latitudes and longitudes into neighbourhood

In [ ]:
cctv['']cctv_df.apply()

In [13]:
# Create list to store neighbourhoods
neighborhood = []
counter = 0

for row in cctv_df:
    counter += 1
    try:
        geolocator = Nominatim(user_agent="Baltimore_Crime")
        neighborhood.append(geolocator.reverse())
    except:
        neighborhood.append(np.NaN)
print(counter)

cctv_df['neighborhood'] = neighborhood


6


ValueError: Length of values does not match length of index